In [1]:
from sqlalchemy import create_engine
from sqlalchemy import text

In [12]:
engine = create_engine("sqlite:///:memory:")

with engine.connect() as con:
    with open("create_schema.sql") as file:
        query_list = file.read().split(';')
        #print(query_list)
        for query_text in query_list:
            query = text(query_text)
            con.execute(query)

In [13]:
from sqlalchemy import MetaData
m = MetaData()
m.reflect(engine)
for table in m.tables.values():
    print(table.name)
    for column in table.c:
        print(column.name)

t_art_objects
ID
DESCRIPTION
PAINTER_ID
CREATION_YEAR
t_art_owners
ID
OWNER_ID
START_OWNERSHIP
END_OWNERSHIP
